## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,37.33,73,33,4.18,scattered clouds
1,1,Busselton,AU,-33.6500,115.3333,46.04,77,0,9.04,clear sky
2,2,Tunceli,TR,39.1083,39.5471,81.46,26,34,8.66,scattered clouds
3,3,Ahipara,NZ,-35.1667,173.1667,51.15,86,2,1.59,clear sky
4,4,Iralaya,HN,15.0000,-83.2333,86.09,78,67,6.42,broken clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Tunceli,TR,39.1083,39.5471,81.46,26,34,8.66,scattered clouds
4,4,Iralaya,HN,15.0000,-83.2333,86.09,78,67,6.42,broken clouds
9,9,Cayenne,GF,4.9333,-52.3333,77.04,88,75,3.44,mist
11,11,Victoria,HK,22.2855,114.1577,86.32,84,20,6.71,light rain
12,12,Kavaratti,IN,10.5669,72.6420,82.42,77,36,12.08,scattered clouds
14,14,Kavieng,PG,-2.5744,150.7967,80.83,77,100,9.17,light rain
16,16,Monrovia,LR,6.3005,-10.7969,80.10,76,97,3.96,overcast clouds
22,22,Atuona,PF,-9.8000,-139.0333,77.68,77,64,22.39,broken clouds
25,25,Kapaa,US,22.0752,-159.3190,77.32,78,75,1.99,broken clouds
32,32,Makakilo City,US,21.3469,-158.0858,75.69,75,1,5.75,clear sky


In [41]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()


City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [42]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [43]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tunceli,TR,81.46,scattered clouds,39.1083,39.5471,
4,Iralaya,HN,86.09,broken clouds,15.0000,-83.2333,
9,Cayenne,GF,77.04,mist,4.9333,-52.3333,
11,Victoria,HK,86.32,light rain,22.2855,114.1577,
12,Kavaratti,IN,82.42,scattered clouds,10.5669,72.6420,
14,Kavieng,PG,80.83,light rain,-2.5744,150.7967,
16,Monrovia,LR,80.10,overcast clouds,6.3005,-10.7969,
22,Atuona,PF,77.68,broken clouds,-9.8000,-139.0333,
25,Kapaa,US,77.32,broken clouds,22.0752,-159.3190,
32,Makakilo City,US,75.69,clear sky,21.3469,-158.0858,


In [44]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
        print("Hotel not found... skipping.")   
        

Hotel not found... skipping.


In [45]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()



In [47]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))